In [1]:
import pandas as pd
import numpy as np
import random
import time
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from keras.layers.advanced_activations import LeakyReLU, PReLU

from sklearn.model_selection import train_test_split 
from sklearn import metrics
from sklearn.utils import class_weight

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import regularizers
from keras.layers import Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

ModuleNotFoundError: No module named 'keras'

In [ ]:
data = pd.read_csv('ks-projects-201801.csv', encoding = 'ISO-8859-1')

In [ ]:
# only use a subset of our whole dataset
length = len(data['goal'])
subset = length - 20000 # if we only want 20,000 rows

data = data.iloc[0:length-subset]

In [215]:
data.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [216]:
# number of days between launch data and deadline
def duration_of_project():
    months[]
    duration = []
    
    for launch, deadline in zip(data['launched'], data['deadline']):
        
        launch = (launch.split())[0]
        launchT = [x.strip() for x in launch.split('-')]
        deadlineT = [x.strip() for x in deadline.split('-')]
    
        year_diff = (int(deadlineT[0]) - int(launchT[0]))*365
        month_diff = (int(deadlineT[1]) - int(launchT[1]))*30
        day_diff = int(deadlineT[2]) - int(launchT[2])
    
        days_offset = year_diff + month_diff + day_diff
        duration.append(days_offset)
        
    data['duration'] = duration

In [217]:
duration_of_project()

In [218]:
# success rate for each main cat
success_rate = {}
success_rate['Publishing'] = 32.04
success_rate['Film & Video'] = 37.42
success_rate['Music'] = 49.72
success_rate['Food'] = 25.07
success_rate['Design'] = 36.84
success_rate['Crafts'] = 24.56
success_rate['Games'] = 38.16
success_rate['Comics'] = 57.01
success_rate['Fashion'] = 26.47
success_rate['Theater'] = 59.76
success_rate['Art'] = 42.47
success_rate['Photography'] = 31.45
success_rate['Technology'] = 20.24
success_rate['Dance'] = 61.82
success_rate['Journalism'] = 22.07

In [219]:
# successful dollars pledged
pos_dollars = {}
pos_dollars['Publishing'] = 136.89
pos_dollars['Film & Video'] = 362.25
pos_dollars['Music'] = 209.19
pos_dollars['Food'] = 123.03
pos_dollars['Design'] = 823.28
pos_dollars['Crafts'] = 14.34
pos_dollars['Games'] = 896.69
pos_dollars['Comics'] = 84.21
pos_dollars['Fashion'] = 143.47
pos_dollars['Theater'] = 38.90
pos_dollars['Art'] = 96.30
pos_dollars['Photography'] = 37.91
pos_dollars['Technology'] = 699.06
pos_dollars['Dance'] = 13.13
pos_dollars['Journalism'] = 13.88

In [ ]:
# number of projects funded (in thousands)
funded = {}
funded['Publishing'] = 14.416
funded['Film & Video'] = 26.172
funded['Music'] = 29.054
funded['Food'] = 6.883
funded['Design'] = 13.141
funded['Crafts'] = 2.483
funded['Games'] = 16.430
funded['Comics'] = 7.511
funded['Fashion'] = 7.213
funded['Theater'] = 6.932
funded['Art'] = 13.981
funded['Photography'] = 3.665
funded['Technology'] = 7.578
funded['Dance'] = 2.505
funded['Journalism'] = 1.166

In [ ]:
# add a column that specifies if a project's goal is above the mean  
# for its category
for cat in data['main_category'].unique():
    print(cat)
    this_cat = np.where(data['main_category'] == cat)
    
    # find mean of the category
    mean_goal = 0
    for row in this_cat[0]:
        mean_goal += data.iloc[row]['goal']
    mean_goal = mean_goal / len(this_cat[0])
    
    for row in this_cat[0]:
        # add column for mean of goal of each category
        data.loc[data.index[row], 'cat_mean'] = mean_goal
        
        # add column for success rate
        data.loc[data.index[row], 'success_rate'] = success_rate[cat]
        
        # add column for positive dollars raised
        data.loc[data.index[row], 'pos_dollars'] = pos_dollars[cat]
        
        # add column for funded projects in category
        data.loc[data.index[row], 'funded_in_cat'] = funded[cat]
        
        # add column for diff of mean and current
        goal_diff = mean_goal - data.iloc[row]['goal']
        data.loc[data.index[row], 'cat_goal_diff'] = goal_diff

Publishing
Film & Video
Music
Food
Design
Crafts
Games
Comics
Fashion
Theater
Art
Photography
Technology


In [ ]:
# same as above but for specific category
pos = 0
neg = 0
for cat in data['category'].unique():
    #print(cat)
    this_cat = np.where(data['category'] == cat)
    
    # find mean of the category
    mean_goal = 0
    mean_backers = 0
    for row in this_cat[0]:
        mean_goal += data.iloc[row]['goal']
        mean_backers += data.iloc[row]['backers']
    mean_goal = mean_goal / len(this_cat[0])
    mean_backers = mean_backers / len(this_cat[0])
    
    for row in this_cat[0]:
        goal_diff = mean_goal - data.iloc[row]['goal']
        data.loc[data.index[row], 'm_cat_goal_diff'] = goal_diff
        
        if data.iloc[row]['backers'] > mean_backers*0.8:
            data.loc[data.index[row], 'mean_backer_binary'] = 1
            pos += 1
        else:
            data.loc[data.index[row], 'mean_backer_binary'] = 0
            neg += 1
            
print(pos)
print(neg)

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
#data['GoalBinary'] = GoalBinary
data['state'].replace(['canceled'], ['failed'], inplace=True)
data['state'].replace(['failed', 'successful'], [-1, 1], inplace=True)

In [ ]:
data['state_binary'] = data['state']
data['state_binary'].replace([-1, 1], [0, 1], inplace=True)

In [ ]:
slim_data = data.copy()
indexNames = slim_data[ (slim_data['state_binary'] != 1) & (slim_data['state_binary'] != 0) ].index
slim_data.drop(indexNames , inplace=True)

In [ ]:
# find length of name
name_length = []
for name in slim_data['name']:
    try:
        name_length.append(len(name))
    except:
        name_length.append(0)
slim_data['name_len'] = name_length

In [ ]:
# scale down data columns
sc = MinMaxScaler()

slim_data['fit_goal'] = sc.fit_transform(slim_data[['goal']])
slim_data['fit_name_len'] = sc.fit_transform(slim_data[['name_len']])
slim_data['fit_duration'] = sc.fit_transform(slim_data[['duration']])
slim_data['fit_cat_mean'] = sc.fit_transform(slim_data[['cat_mean']])
slim_data['fit_goal_diff'] = sc.fit_transform(slim_data[['cat_goal_diff']])
slim_data['fit_m_cat_goal'] = sc.fit_transform(slim_data[['m_cat_goal_diff']])
slim_data['fit_s_rate'] = sc.fit_transform(slim_data[['success_rate']])
slim_data['fit_pos_dollars'] = sc.fit_transform(slim_data[['pos_dollars']])
slim_data['fit_funded'] = sc.fit_transform(slim_data[['funded_in_cat']])

In [ ]:
training=pd.get_dummies(slim_data, columns=["main_category"])
training=pd.get_dummies(training, columns=["category"])
training=pd.get_dummies(training, columns=["country"])
training=pd.get_dummies(training, columns=["currency"])

#y = slim_data['state_binary']
y = slim_data['mean_backer_binary']
X = training.loc[:, 'fit_goal':]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
# build DNN

model = Sequential()  # sequential layered network
dims = len(X.columns) # dimension of the input data

class_weights = class_weight.compute_class_weight('balanced', 
                                                  np.unique(y_train), 
                                                  y_train)

# input layer
model.add(Dense(100, activation='linear', input_dim = dims))
model.add(LeakyReLU(alpha=.001))
model.add(Dropout(0.5))

# second layer
model.add(Dense(100, activation='linear'))
model.add(LeakyReLU(alpha=.001))
model.add(Dropout(0.5))


# third layer
model.add(Dense(100, activation='linear'))
model.add(LeakyReLU(alpha=.001))
model.add(Dropout(0.5))


# fourth layer
model.add(Dense(85, activation='linear'))
model.add(LeakyReLU(alpha=.001))
model.add(Dropout(0.5))


# fith layer
model.add(Dense(75, activation='linear'))
model.add(LeakyReLU(alpha=.001))
model.add(Dropout(0.5))

# output layer
model.add(Dense(1, activation='sigmoid'))

# compile the model using rmsprop opt and mse loss
model.compile(optimizer = 'adam',  
              loss = 'mse',
              metrics = ['accuracy'])

# Train the model, iterating on the data in batches of 32 samples
history = model.fit(X_train, y_train, 
                    validation_split = 0.33, 
                    epochs = 20,
                    class_weight = class_weights,
                    batch_size = 35)

# Evaluate the model's performance
train_loss, train_acc = model.evaluate(X_train, y_train)
test_loss, test_acc = model.evaluate(X_test, y_test)

print('Training set accuracy:', train_acc)
print('Test set accuracy:', test_acc)

In [ ]:
# The history of our accuracy during training.
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Number of epochs')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
backer_mean = model.predict(X)

X2 = X.copy()
X2['backer_mean_pred'] = backer_mean

y2 = slim_data['state_binary']
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size = 0.2, random_state=0)
X2.head()

In [ ]:
# build DNN

model2 = Sequential()  # sequential layered network
dims = len(X2.columns) # dimension of the input data

# input layer
model2.add(Dense(100, activation='linear', input_dim = dims))
model2.add(LeakyReLU(alpha=.001))
model2.add(Dropout(0.5))

# second layer
model2.add(Dense(100, activation='linear'))
model2.add(LeakyReLU(alpha=.001))
model2.add(Dropout(0.5))


# third layer
model2.add(Dense(100, activation='linear'))
model2.add(LeakyReLU(alpha=.001))
model2.add(Dropout(0.5))


# fourth layer
model2.add(Dense(85, activation='linear'))
model2.add(LeakyReLU(alpha=.001))
model2.add(Dropout(0.5))


# fith layer
model2.add(Dense(75, activation='linear'))
model2.add(LeakyReLU(alpha=.001))
model2.add(Dropout(0.5))

# output layer
model2.add(Dense(1, activation='sigmoid'))

# compile the model using rmsprop opt and mse loss
model2.compile(optimizer = 'adam',  
              loss = 'mse',
              metrics = ['accuracy'])

# Train the model, iterating on the data in batches of 32 samples
history = model2.fit(X2_train, y2_train, 
                    validation_split = 0.33, 
                    epochs = 50,
                    batch_size = 35)

# Evaluate the model's performance
train_loss, train_acc = model2.evaluate(X2_train, y2_train)
test_loss, test_acc = model2.evaluate(X2_test, y2_test)

print('Training set accuracy:', train_acc)
print('Test set accuracy:', test_acc)

In [ ]:
# The history of our accuracy during training.
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Number of epochs')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# The history of our cross-entropy loss during training.
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Number of epochs')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
predictions = model2.predict(X2_test)
y_pred = []
neg = pos = 0
for pred in predictions:
    if pred < 0.5:
        neg += 1
        y_pred.append(0.0)
    else:
        pos += 1
        y_pred.append(1.0)
print("pos:", pos, " | neg:", neg)
print(len(X_test))

In [ ]:
y_true = []
for val in y2_test:
    y_true.append(val)
matrix = confusion_matrix(y_true, y_pred)
print(matrix)